In [ ]:
## Google drive mounting
from google.colab import auth
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
#file download from the google drive
# folder id
folder_id = '13HMnXVHuRSz7HxZn80gh_N7VEWC4TlLY'

# downloading of the file
file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

for file in file_list:
    print(f'Downloading {file["title"]}...')
    file.GetContentFile(file['title'])

In [ ]:
## data preprocessing
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive',force_remount='True')

# Import necessary libraries
import pandas as pd
import mne

# Assuming you have already authenticated and created the PyDrive client
# Authenticate and create the PyDrive client (if not already done)
from google.colab import auth
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Replace 'your_folder_id' with the actual folder ID of your EEG data folder
folder_id = '13HMnXVHuRSz7HxZn80gh_N7VEWC4TlLY'

# List and download files from the folder
file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

data_frames = []
for file in file_list:
    df = pd.read_csv(file['title'])
    data_frames.append(df)

# Combine data from all files
combined_data = pd.concat(data_frames, ignore_index=True)

# Display the first few rows of the combined data
print(combined_data.head())

# Assuming the EEG data has columns for time and channels like Fp1, Fp2, etc.
# Exclude the time column and convert to numeric
data = combined_data.iloc[:, 1:].apply(pd.to_numeric, errors='coerce').values.T  # Transpose to get channels as rows
ch_names = combined_data.columns[1:].tolist()  # Convert to list
info = mne.create_info(ch_names=ch_names, sfreq=256, ch_types='eeg')  # Adjust sfreq as needed
raw = mne.io.RawArray(data, info)

# Preprocessing steps

# 1. Filtering (e.g., 1-50 Hz band-pass filter)
raw.filter(1., 50., fir_design='firwin')

# 2. Notch filter to remove power line noise (e.g., 50 Hz in Europe, 60 Hz in the US)
raw.notch_filter(freqs=50)

# Save the preprocessed data to a CSV file
preprocessed_data_df = pd.DataFrame(data.T, columns=ch_names)
csv_path = '/content/drive/MyDrive/preprocessed_eeg_data.csv'
preprocessed_data_df.to_csv(csv_path, index=False)

print(f"Preprocessing complete. Data saved as 'preprocessed_eeg_data.fif' and 'preprocessed_eeg_data.csv' in Google Drive at {csv_path}")


In [ ]:
## Feature extraction
# Import necessary libraries
import numpy as np
from scipy.signal import welch

# Path to the CSV file in Google Drive
csv_path = '/content/drive/MyDrive/preprocessed_eeg_data.csv'

# Load the CSV file into a DataFrame
data = pd.read_csv(csv_path)

# Display the first few rows of the loaded data
print(data.head())


# Define frequency bands
bands = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 12),
    'beta': (12, 30),
    'gamma': (30, 45)
}

# Initialize a dictionary to store the features
features = {ch: {} for ch in raw.ch_names}

# Calculate Power Spectral Density (PSD) for each channel
data_array = raw.get_data()
psds = []
freqs = None
for ch_data in data_array:
    f, pxx = welch(ch_data, fs=raw.info['sfreq'], nperseg=256)
    psds.append(pxx)
    if freqs is None:
        freqs = f
psds = np.array(psds)

# Extract band power for each frequency band
for ch_idx, ch_name in enumerate(raw.ch_names):
    for band, (low, high) in bands.items():
        # Select the frequency range for the current band
        freq_mask = (freqs >= low) & (freqs <= high)
        # Calculate the band power using the trapezoidal rule
        band_power = np.trapz(psds[ch_idx][freq_mask], freqs[freq_mask])
        features[ch_name][f'{band}_power'] = band_power

# Extract statistical features (mean, variance)
for ch_idx, ch_name in enumerate(raw.ch_names):
    features[ch_name]['mean'] = np.mean(data_array[ch_idx])
    features[ch_name]['variance'] = np.var(data_array[ch_idx])

# Convert the features dictionary to a DataFrame for easier analysis
feature_df = pd.DataFrame(features).T
print(feature_df)

# Save the features DataFrame to a CSV file in Google Drive
feature_csv_path = '/content/drive/MyDrive/eeg_features.csv'
feature_df.to_csv(feature_csv_path, index=True)

print(f"Feature extraction complete. Features saved as 'eeg_features.csv' in Google Drive at {feature_csv_path}")

In [ ]:
## model training from generative adversal network and auto encoder
# Import necessary libraries
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

# Load the preprocessed EEG data from CSV file
csv_path = '/content/drive/MyDrive/preprocessed_eeg_data.csv'
data = pd.read_csv(csv_path)

# Display the first few rows of the loaded data
print(data.head())

# Assuming the EEG data has columns for time and channels like Fp1, Fp2, etc.
# Exclude the time column and convert to numeric
data_values = data.iloc[:, 1:].apply(pd.to_numeric, errors='coerce').values.T
ch_names = data.columns[1:].tolist()
info = mne.create_info(ch_names=ch_names, sfreq=256, ch_types='eeg')
raw = mne.io.RawArray(data_values, info)

# Normalize the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_values.T).T

# Convert to DataFrame for easier handling
df = pd.DataFrame(data_scaled.T, columns=ch_names)

# Autoencoder architecture
input_dim = df.shape[1]
encoding_dim = 64  # Adjust based on your requirement

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation='relu')(input_layer)
decoder = Dense(input_dim, activation='sigmoid')(encoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mse')

# Train the autoencoder
autoencoder.fit(df, df, epochs=25, batch_size=32, shuffle=True)

# Encode the data
encoded_data = autoencoder.predict(df)

# GAN architecture
def build_generator(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(output_dim, activation='sigmoid'))
    return model

def build_discriminator(input_dim):
    model = Sequential()
    model.add(Dense(256, input_dim=input_dim, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

input_dim_gan = 100
output_dim_gan = df.shape[1]

generator = build_generator(input_dim_gan, output_dim_gan)
discriminator = build_discriminator(output_dim_gan)
discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])

discriminator.trainable = False
gan_input = Input(shape=(input_dim_gan,))
gen_out = generator(gan_input)
gan_out = discriminator(gen_out)
gan = Model(gan_input, gan_out)
gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')

# Training the GAN
epochs = 5000
batch_size = 64

for epoch in range(epochs):
    # Train discriminator
    idx = np.random.randint(0, df.shape[0], batch_size)
    real_samples = df.values[idx]
    noise = np.random.normal(0, 1, (batch_size, input_dim_gan))
    generated_samples = generator.predict(noise)

    d_loss_real = discriminator.train_on_batch(real_samples, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(generated_samples, np.zeros((batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train generator
    noise = np.random.normal(0, 1, (batch_size, input_dim_gan))
    valid_y = np.ones((batch_size, 1))
    g_loss = gan.train_on_batch(noise, valid_y)

    if epoch % 100 == 0:
        print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100 * d_loss[1]}%] [G loss: {g_loss}]")

print("GAN training complete.")
